In [47]:
#preprocessing of the data

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
df = pd.read_csv('../data/compas-scores-two-years.csv')
features = ['race', 'age', 'sex', 'juv_misd_count', 'priors_count']
#features chosen for our C-Logistic Regression
to_predict = 'two_year_recid'
races_to_filter = ['Caucasian', 'African-American']
filtered = df.loc[df['race'].isin(races_to_filter), features + [to_predict]].reset_index(drop=True)

#replace categorical data with boolean numbers, 0 and 1
filtered['race'] = filtered['race'].apply(lambda race: 0 if race == 'Caucasian' else 1)
filtered['sex'] = filtered['sex'].apply(lambda sex: 0 if sex == 'Male' else 1)
#x=filtered[['race', 'age', 'sex', 'juv_misd_count', 'priors_count']]
#y=filtered[['two_year_recid']]
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#Normalizing the data, so each variable is similar in weight
normalized_df = (filtered-filtered.mean())/filtered.std()
filtered['age'] = normalized_df['age']
filtered['juv_misd_count'] = normalized_df['juv_misd_count']
filtered['priors_count'] = normalized_df['priors_count']

In [81]:
#Logistic Regression

#We adapted the code from the repository linked with our paper "Fairness Constraints: Mechanisms for Fair Classification"
#utils, the file that we are importing the train_model function from is also in the doc folder of our repository

from utils import train_model
import loss_funcs

train_size = 5000
x_train = filtered.loc[:train_size, features]
y_train = filtered.loc[:train_size, to_predict]
x_test = filtered.loc[train_size:, features]
y_test = filtered.loc[train_size:, to_predict]
x_control = {'race': x_train['race'].to_list()}



apply_fairness_constraints = 0
apply_accuracy_constraint = 0
sep_constraint = 0
gamma = 0
sensitive_attrs = []
sensitive_attrs_to_cov_thresh = {}

#coefficients from training the model
w = train_model(x_train.to_numpy(),
                y_train.to_numpy(),
                x_control,
                loss_funcs._logistic_loss,
                apply_fairness_constraints,
                apply_accuracy_constraint,
                sep_constraint,
                sensitive_attrs,
                sensitive_attrs_to_cov_thresh,
                gamma)
w

array([22.33301874, -0.06273929, 18.45664809, -0.33452549,  0.16826391])

In [82]:
from sklearn.metrics import classification_report, confusion_matrix

m = LogisticRegression()
m.coef_= w.reshape((1,-1))
m.intercept_ = 0
m.classes_ = np.array([0, 1])
y_pred = m.predict(x_test[features])
print(classification_report(y_test, y_pred))
compute_p_rule(x_test['race'], y_pred)

              precision    recall  f1-score   support

           0       0.70      0.21      0.33       609
           1       0.50      0.90      0.64       541

    accuracy                           0.53      1150
   macro avg       0.60      0.55      0.49      1150
weighted avg       0.61      0.53      0.48      1150


Total data points: 1150
# non-protected examples: 697
# protected examples: 453
Non-protected in positive class: 697 (100%)
Protected in positive class: 267 (59%)
P-rule is: 59%


/home/mpark/.local/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


58.94039735099338

In [87]:
#section applying fairness constraints

apply_fairness_constraints = 1
apply_accuracy_constraint = 0
sep_constraint = 0
gamma = 0
sensitive_attrs = ['race']
sensitive_attrs_to_cov_thresh = {'race': 0}

w = train_model(x_train.to_numpy(),
                y_train.to_numpy(),
                x_control,
                loss_funcs._logistic_loss,
                apply_fairness_constraints,
                apply_accuracy_constraint,
                sep_constraint,
                sensitive_attrs,
                sensitive_attrs_to_cov_thresh,
                gamma)
w

array([ 2.20439424e+01, -6.26961949e-02,  3.76846962e+02, -3.34555972e-01,
        1.68283817e-01])

In [88]:
from sklearn.metrics import classification_report, confusion_matrix

m = LogisticRegression()
m.coef_= w.reshape((1,-1))
m.intercept_ = 0
m.classes_ = np.array([0, 1])
y_pred = m.predict(x_test[features])
print(classification_report(y_test, y_pred))
compute_p_rule(x_test['race'], y_pred)

              precision    recall  f1-score   support

           0       0.70      0.21      0.33       609
           1       0.50      0.90      0.64       541

    accuracy                           0.53      1150
   macro avg       0.60      0.55      0.49      1150
weighted avg       0.61      0.53      0.48      1150


Total data points: 1150
# non-protected examples: 697
# protected examples: 453
Non-protected in positive class: 697 (100%)
Protected in positive class: 267 (59%)
P-rule is: 59%


/home/mpark/.local/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


58.94039735099338

In [13]:
import utils as ut
import loss_funcs as lf
def test_data():
    X, y, x_control = filtered
    ut.compute_p_rule(x_control["sex"], y) # compute the p-rule in the original data
    
    """ Split the data into train and test """
    X = ut.add_intercept(X) # add intercept to X before applying the linear classifier
    train_fold_size = 0.7
    x_train, y_train, x_control_train, x_test, y_test, x_control_test = ut.split_into_train_test(X, y, x_control, train_fold_size)
    
    apply_fairness_constraints = None
    apply_accuracy_constraint = None
    sep_constraint = None

    loss_function = lf._logistic_loss
    sensitive_attrs = ["sex"]
    sensitive_attrs_to_cov_thresh = {}
    gamma = None
    
    def train_test_classifier():
        w = ut.train_model(x_train, y_train, x_control_train, loss_function, apply_fairness_constraints, apply_accuracy_constraint, sep_constraint, sensitive_attrs, sensitive_attrs_to_cov_thresh, gamma)
        train_score, test_score, correct_answers_train, correct_answers_test = ut.check_accuracy(w, x_train, y_train, x_test, y_test, None, None)
        distances_boundary_test = (np.dot(x_test, w)).tolist()
        all_class_labels_assigned_test = np.sign(distances_boundary_test)
        correlation_dict_test = ut.get_correlations(None, None, all_class_labels_assigned_test, x_control_test, sensitive_attrs)
        cov_dict_test = ut.print_covariance_sensitive_attrs(None, x_test, distances_boundary_test, x_control_test, sensitive_attrs)
        p_rule = ut.print_classifier_fairness_stats([test_score], [correlation_dict_test], [cov_dict_test], sensitive_attrs[0])	
        return w, p_rule, test_score
    
    print 
    print("== Unconstrained (original) classifier ==")
    # all constraint flags are set to 0 since we want to train an unconstrained (original) classifier
    apply_fairness_constraints = 0
    apply_accuracy_constraint = 0
    sep_constraint = 0
    w_uncons, p_uncons, acc_uncons = train_test_classifier()
    
    """ Now classify such that we optimize for accuracy while achieving perfect fairness """
    apply_fairness_constraints = 1 # set this flag to one since we want to optimize accuracy subject to fairness constraints
    apply_accuracy_constraint = 0
    sep_constraint = 0
    sensitive_attrs_to_cov_thresh = {"sex":0}
    print
    print("== Classifier with fairness constraint ==")
    w_f_cons, p_f_cons, acc_f_cons  = train_test_classifier()
    
    
    """ Classify such that we optimize for fairness subject to a certain loss in accuracy """
    apply_fairness_constraints = 0 # flag for fairness constraint is set back to0 since we want to apply the accuracy constraint now
    apply_accuracy_constraint = 1 # now, we want to optimize fairness subject to accuracy constraints
    sep_constraint = 0
    gamma = 0.5 # gamma controls how much loss in accuracy we are willing to incur to achieve fairness -- increase gamme to allow more loss in accuracy
    print("== Classifier with accuracy constraint ==")
    w_a_cons, p_a_cons, acc_a_cons = train_test_classifier()	
    
    """ 
    Classify such that we optimize for fairness subject to a certain loss in accuracy 
    In addition, make sure that no points classified as positive by the unconstrained (original) classifier are misclassified.
    """
    apply_fairness_constraints = 0 # flag for fairness constraint is set back to0 since we want to apply the accuracy constraint now
    apply_accuracy_constraint = 1 # now, we want to optimize accuracy subject to fairness constraints
    sep_constraint = 1 # set the separate constraint flag to one, since in addition to accuracy constrains, we also want no misclassifications for certain points (details in demo README.md)
    gamma = 1000.0
    print("== Classifier with accuracy constraint (no +ve misclassification) ==")
    w_a_cons_fine, p_a_cons_fine, acc_a_cons_fine  = train_test_classifier()
    
    return



In [89]:
#SVM

#Preproccessing

features = ['race', 'age', 'sex', 'juv_misd_count', 'priors_count']
to_predict = 'two_year_recid'
races_to_filter = ['Caucasian', 'African-American']
# df.loc[df['race'].isin(races_to_filter), features + [to_predict]]
df = df.loc[df['race'].isin(races_to_filter), features + [to_predict]]


#transform race and sex into 0 and 1 
#African-American will be 0 and Caucasian will be 1
#Male will be 0 and Female will be 1

df['race'] = df['race'].replace(['African-American'],0)
df['race'] = df['race'].replace(['Caucasian'],1)
df['sex'] = df['sex'].replace(['Male'],0)
df['sex'] = df['sex'].replace(['Female'],1)

#normalize age, juv_misd_count, and priors_count

normalized_df = (df-df.mean())/df.std()
df['age'] = normalized_df['age']
df['juv_misd_count'] = normalized_df['juv_misd_count']
df['priors_count'] = normalized_df['priors_count']
# normalized_df.head()
df.head()


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X = df.drop('two_year_recid',axis=1)
Y = df['two_year_recid']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20)

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

#Add Fairness Constraints and p% evaluation
compute_p_rule(X_test['race'], y_pred)

[[553 114]
 [331 232]]
              precision    recall  f1-score   support

           0       0.63      0.83      0.71       667
           1       0.67      0.41      0.51       563

    accuracy                           0.64      1230
   macro avg       0.65      0.62      0.61      1230
weighted avg       0.65      0.64      0.62      1230


Total data points: 1230
# non-protected examples: 504
# protected examples: 726
Non-protected in positive class: 90 (18%)
Protected in positive class: 256 (35%)
P-rule is: 197%


197.46556473829202